In [1]:
from scipy import io
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation
from scipy.linalg import sqrtm
from quaternion import Quaternion
import math
from estimate_rot import estimate_rot

In [2]:
data_num = 1
imu = io.loadmat('imu/imuRaw'+str(data_num)+'.mat')
accel = imu['vals'][0:3,:] # order: roll, pitch, yaw
gyro = imu['vals'][3:6,:]
T = np.shape(imu['ts'])[1]
ts_imu = imu['ts'].reshape(-1,)
angle_names = ['roll', 'pitch', 'yaw']


accel_bias, accel_sensitivity = accel_calib_params(accel)
accel = accel_calibration(accel, accel_bias, accel_sensitivity)

gyro_bias, gyro_sensitivity = gyro_calib_params(gyro)
gyro = gyro_calibration(gyro, gyro_bias, gyro_sensitivity)

vicon = io.loadmat('vicon/viconRot'+str(data_num)+'.mat')
T_vicon = np.shape(vicon['ts'])[1]
ts_vicon = vicon['ts'].reshape(-1,)

NameError: name 'accel_calib_params' is not defined

In [ ]:
euler_vicon = []
for t in range(T_vicon):
    vicon_r = Rotation.from_matrix(vicon['rots'][:,:,t])
    euler_vicon.append(vicon_r.as_euler("zyx")) #yaw, pitch, roll
euler_vicon = np.array(euler_vicon).T
euler_vicon[[2,0], :] = euler_vicon[[0,2], :]

# plt.figure(figsize = (10,3))
# for i in range(3):
#     plt.plot(ts_vicon, euler_vicon[i,:], label = angle_names[i])
# plt.legend()
# plt.title('Vicon Euler angles data')

### Initialization
choose the values of the initial covariance of the state, dynamics noise and measurement noise

In [ ]:
seed = 1212
np.random.seed(seed)
n = 6
# state covariance initialized
cov0 = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5]) # 6-dimension vector, similar dimension with process noise vector w_k =(w_{quaternion}, w_{angular velo})

cov_k_k = np.diag(cov0) #6x6 matrix

# initialize mu state
mu0 = np.array([[0.5, 0., -0.6, 1, 1, 1]]).T # mean vector of orientation data and angular velo
Q_mu = Quaternion()
Q_mu.from_axis_angle(mu0[:3].reshape(-1))
mu_k_k = (Q_mu.q, mu0[3], mu0[4], mu0[5]) # a 7-d vector (first 4 for quaternion, remaining 3 for angular velo)

# process noise/dynamic noise
R = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
R = np.diag(R)
dt = 0.01 # this varies. Replace by t_{k+1} - t_k in the loop

# measurement noise:
Q = ...

# initial state
# X0 = [float(np.random.normal(mu0[i], np.sqrt(cov0[i]), 1)) for i in range(n)]
# Q_state = Quaternion()
# Q_state.from_axis_angle(X0[:3])
# w0_x, w0_y, w0_z = 0, 0, 0
# state = (Q_state.q, w0_x, w0_y, w0_z)

### Generate sigma points 
 
Given n= 6 => we will have 12 sigma points

First computing square root of covariance 

Then transform first 3 elements to quaternion (axis-angle form), before "adding" to mu_x (quaternion multiplication for first 4 elements and pure addition for 3 angular velo elements)




In [ ]:
# first compute the square root
sqrt_cov = sqrtm(cov_k_k + R*dt)
sqrt_cov_cols = np.concatenate((sqrt_cov * np.sqrt(n), sqrt_cov * (-np.sqrt(n))), axis = 1)

sigma_q = []

# q_bar can be initialized from quaternion of the previous state ( mu_{k|k}). Will do this in the loop
Q_bar = Quaternion(scalar = mu_k_k[0][0], vec = mu_k_k[0][1:4] )
# Q_bar = Quaternion()
# Q_bar = Q_state
error = np.zeros((3, 2*n))

for i in range(2*n):
    Q_sigma = Quaternion( ) #convert first 3 elements of col vector to axis-angle
    Q_sigma.from_axis_angle(sqrt_cov_cols[:3, i])


    # Adding to mu_x
    quat = Q_bar * Q_sigma # multiply axis-angle form of square root of covariance with the quaternion element in the state
    sigma_q.append( quat.q )
    
    # Gradient descent
    Q_error = quat * Q_bar.inv()
    Q_error.normalize()
    error[:,i] = Q_error.axis_angle() # convert error vector to axis-angle representation

error_bar = np.sum(error, axis = 1)/ (2*n)
Q_error_bar = Quaternion()
Q_error_bar.from_axis_angle(error_bar)
Q_bar = Q_error_bar * Q_bar

for iter in range(100):
    error = np.zeros((3, 2*n))
    for i in range(2*n):
        quat = Quaternion(scalar=sigma_q[i][0], vec=sigma_q[i][1:4])
        # print(quat)
        Q_error = quat * Q_bar.inv()
        Q_error.normalize()
        # print(Q_error)
        error[:,i] = Q_error.axis_angle()
    # print(error)
    error_bar = np.sum(error, axis = 1)/ (2*n)
    # print(error_bar)
    error_norm = np.linalg.norm(error_bar)
    print(error_norm)

    Q_error_bar.from_axis_angle(error_bar)
    Q_bar = Q_error_bar * Q_bar 
weight = 1/(2*n)



2.8609792490763984e-17
2.8609792490763984e-17
2.8609792490763984e-17
3.0068540250264654e-17
3.1031676915590914e-17
1.6679025177028005e-17
1.6679025177028005e-17
5.171946152598485e-17
2.8609792490763984e-17
4.944553638389996e-17
2.1820464722267795e-17
2.1820464722267795e-17
3.1031676915590914e-17
2.1820464722267795e-17
2.1820464722267795e-17
3.0068540250264654e-17
1.976197850111478e-17
2.1820464722267795e-17
3.0068540250264654e-17
1.976197850111478e-17
2.1820464722267795e-17
3.0068540250264654e-17
1.976197850111478e-17
2.1820464722267795e-17
3.0068540250264654e-17
1.976197850111478e-17
2.1820464722267795e-17
3.0068540250264654e-17
1.976197850111478e-17
2.1820464722267795e-17
3.0068540250264654e-17
1.976197850111478e-17
2.1820464722267795e-17
3.0068540250264654e-17
1.976197850111478e-17
2.1820464722267795e-17
3.0068540250264654e-17
1.976197850111478e-17
2.1820464722267795e-17
3.0068540250264654e-17
1.976197850111478e-17
2.1820464722267795e-17
3.0068540250264654e-17
1.976197850111478e-17


q_bar is new state mean of the quaternion part of sigma points.

Now calculate covariance for quaternion of each sigma point

In [ ]:
sigma_cov_q = np.zeros((3,3))
for i in range(2*n):
    sigma_cov_q += (error[:,i] - error_bar).reshape(3,1) @ (error[:,i] - error_bar).reshape(1,3) /(2*n)
sigma_cov_q

array([[ 5.01000000e-01,  2.08166817e-17, -3.46944695e-17],
       [ 2.08166817e-17,  5.01000000e-01, -2.77555756e-17],
       [-3.46944695e-17, -2.77555756e-17,  5.01000000e-01]])

The estimate of the mean and covariance for the angular velocity, which is a Euclidean vector, are obtained in the standard way

In [ ]:
sigma_w = np.array([np.array(mu_k_k[1:4]).reshape(-1) + sqrt_cov_cols[3:, 0] for i in range(2*n)]).T #state in here refers to mu_k|k

# mu
sigma_mu_w = np.array([ np.sum(weight * (mu_k_k[j+1] + sigma_w[j, :])) for j in range(3)]) 
sigma_cov_w = np.zeros((3,3))
for i in range(12):
    diff = ((np.array(mu_k_k[1:4]).reshape(-1) + sigma_w[:,i]) - sigma_mu_w).reshape(3,1)
    sigma_cov_w += weight * (diff @ diff.T)

In [ ]:
# Predicted mu and covariance of state
mu_k1_k = (Q_bar.q, sigma_mu_w[0], sigma_mu_w[1], sigma_mu_w[2])

cov_k1_k = np.zeros((6,6))
cov_k1_k[:3, :3] = sigma_cov_q
cov_k1_k[3:, 3:] = sigma_cov_w

## Now, MEASUREMENT UPDATE

# Test estimate_rot file

0
[0.2 0.2 0.2]
[0.4 0.4 0.4]
1
[0.2 0.2 0.2]
[0.4 0.4 0.4]
2
[0.2 0.2 0.2]
[0.4 0.4 0.4]
3
[0.2 0.2 0.2]
[0.4 0.4 0.4]
4
[0.2 0.2 0.2]
[0.4 0.4 0.4]
5
[0.2 0.2 0.2]
[0.4 0.4 0.4]
6
[0.2 0.2 0.2]
[0.4 0.4 0.4]
7
[0.2 0.2 0.2]
[0.4 0.4 0.4]
8
[0.2 0.2 0.2]
[0.4+0.j 0.4+0.j 0.4+0.j]


/Users/tuna28ng/ESE650/hw2/p2/quaternion.py:41: ComplexWarning: Casting complex values to real discards the imaginary part
  self.q[1:4] = axis*math.sin(angle/2)


UFuncTypeError: Cannot cast ufunc 'add' output from dtype('complex128') to dtype('float64') with casting rule 'same_kind'